In [1]:
%cd ..

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark


In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib nbagg
import os.path as op
import time

from keras.backend.tensorflow_backend import set_session
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau, LearningRateScheduler
from keras.models import load_model
from keras.utils.vis_utils import model_to_dot
from keras_tqdm import TQDMNotebookCallback
from keras import backend as K
import tensorflow as tf
from tqdm import tqdm_notebook

from data import MaskShifted2DSequence, MaskShiftedSingleImage2DSequence,MaskedUntouched2DSequence
from evaluate import psnr, ssim
from pdnet import pdnet, invnet
from pdnet_crop import pdnet_crop, invnet_crop

/volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/venv/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
tf.logging.set_verbosity(tf.logging.INFO)
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
                                    # (nothing gets printed in Jupyter, only if you run it standalone)
sess = tf.Session(config=config)
set_session(sess)  # set this TensorFlow session as the default session for Keras

In [4]:
# paths
train_path = '/media/Zaccharie/UHRes/singlecoil_train/singlecoil_train/'
val_path = '/media/Zaccharie/UHRes/singlecoil_val/'
test_path = '/media/Zaccharie/UHRes/singlecoil_test/'

In [5]:
n_samples_train = 34742
n_samples_val = 7135

n_volumes_train = 973
n_volumes_val = 199

In [6]:
# generators
AF = 4
# MaskShifted2DSequence, MaskShiftedSingleImage2DSequence, MaskedUntouched2DSequence
train_gen = MaskedUntouched2DSequence(train_path, af=AF, inner_slices=1)
val_gen = MaskedUntouched2DSequence(val_path, af=AF)

In [7]:
run_params = {
    'n_primal': 5,
    'n_dual': 5,
    'n_iter': 10,
    'n_filters': 32,
    'res_connection': False,
    'primal_only': False,
}

n_epochs = 10
run_id = f'pdnet_af{AF}_{int(time.time())}'
chkpt_path = f'checkpoints/{run_id}' + '-{epoch:02d}.hdf5'
print(run_id)

In [8]:
chkpt_cback = ModelCheckpoint(chkpt_path, period=2, save_weights_only=True)
log_dir = op.join('logs', run_id)
tboard_cback = TensorBoard(
    log_dir=log_dir, 
    histogram_freq=0, 
    write_graph=True, 
    write_images=False, 
)
lr_on_plat_cback = ReduceLROnPlateau(monitor='val_loss', min_lr=5*1e-5, mode='auto', patience=3)
tqdm_cb = TQDMNotebookCallback(metric_format="{name}: {value:e}")

In [9]:
model = pdnet_crop(lr=1e-3, **run_params)
print(model.summary(line_length=150))

W0827 15:29:27.416386 140290819348224 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:529: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0827 15:29:27.428352 140290819348224 deprecation.py:323] From /volatile/home/Zaccharie/workspace/fastmri-reproducible-benchmark/pdnet_crop.py:70: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0827 15:29:27.487764 140290819348224 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0827 15:29:27.509011 140290819348224 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:4467: The name tf.truncated_normal is deprecated. Please use tf.ra

Model: "model_1"
______________________________________________________________________________________________________________________________________________________
Layer (type)                                     Output Shape                     Param #           Connected to                                      
kspace_input (InputLayer)                        (None, 640, None, 1)             0                                                                   
______________________________________________________________________________________________________________________________________________________
buffer_primal (Lambda)                           (None, 640, None, 5)             0                 kspace_input[0][0]                                
______________________________________________________________________________________________________________________________________________________
mask_input (InputLayer)                          (None, 640, None)           

In [10]:
# model.fit_generator(
#     train_gen, 
#     steps_per_epoch=10, 
#     epochs=n_epochs,
#     validation_data=val_gen,
#     validation_steps=10,
#     verbose=0,
#     callbacks=[tqdm_cb, tboard_cback, chkpt_cback, lr_on_plat_cback],
#     max_queue_size=100,
#     use_multiprocessing=True,
#     workers=35,
# )

In [11]:
%%time
# simple overfit trials
data = train_gen[0]
val_data = val_gen[0]
model.fit(
    x=data[0], 
    y=data[1], 
#     validation_data=val_data, 
    batch_size=data[0][0].shape[0], 
    epochs=500,
    verbose=2, 
    shuffle=False,
)

W0827 15:29:30.895118 140290819348224 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1021: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.

W0827 15:29:31.265439 140290819348224 deprecation_wrapper.py:119] From /volatile/home/Zaccharie/workspace/keras/keras/backend/tensorflow_backend.py:1008: The name tf.assign is deprecated. Please use tf.compat.v1.assign instead.



Epoch 1/500
 - 6s - loss: 0.0010 - mean_squared_error: 1.7102e-06 - keras_psnr: -3.8980e+00 - keras_ssim: 0.0012
Epoch 2/500
 - 0s - loss: 0.0011 - mean_squared_error: 1.8489e-06 - keras_psnr: -4.2367e+00 - keras_ssim: 0.0027
Epoch 3/500
 - 0s - loss: 7.0459e-04 - mean_squared_error: 7.4991e-07 - keras_psnr: -3.1756e-01 - keras_ssim: 0.0074
Epoch 4/500
 - 0s - loss: 4.1144e-04 - mean_squared_error: 2.6847e-07 - keras_psnr: 4.1437 - keras_ssim: 0.0092
Epoch 5/500
 - 0s - loss: 3.1199e-04 - mean_squared_error: 1.4418e-07 - keras_psnr: 6.8435 - keras_ssim: 0.0117
Epoch 6/500
 - 0s - loss: 4.0146e-04 - mean_squared_error: 2.9426e-07 - keras_psnr: 3.7452 - keras_ssim: 0.0077
Epoch 7/500
 - 0s - loss: 2.8116e-04 - mean_squared_error: 1.6617e-07 - keras_psnr: 6.2271 - keras_ssim: 0.0186
Epoch 8/500
 - 0s - loss: 1.5400e-04 - mean_squared_error: 4.3565e-08 - keras_psnr: 12.0412 - keras_ssim: 0.0344
Epoch 9/500
 - 0s - loss: 1.3348e-04 - mean_squared_error: 3.2925e-08 - keras_psnr: 13.2573 - ke

Epoch 73/500
 - 0s - loss: 1.1655e-05 - mean_squared_error: 2.6150e-10 - keras_psnr: 34.2578 - keras_ssim: 0.7658
Epoch 74/500
 - 0s - loss: 1.1599e-05 - mean_squared_error: 2.5858e-10 - keras_psnr: 34.3066 - keras_ssim: 0.7677
Epoch 75/500
 - 0s - loss: 1.1547e-05 - mean_squared_error: 2.5578e-10 - keras_psnr: 34.3539 - keras_ssim: 0.7694
Epoch 76/500
 - 0s - loss: 1.1497e-05 - mean_squared_error: 2.5309e-10 - keras_psnr: 34.3999 - keras_ssim: 0.7711
Epoch 77/500
 - 0s - loss: 1.1450e-05 - mean_squared_error: 2.5042e-10 - keras_psnr: 34.4459 - keras_ssim: 0.7726
Epoch 78/500
 - 0s - loss: 1.1403e-05 - mean_squared_error: 2.4778e-10 - keras_psnr: 34.4919 - keras_ssim: 0.7740
Epoch 79/500
 - 0s - loss: 1.1356e-05 - mean_squared_error: 2.4520e-10 - keras_psnr: 34.5374 - keras_ssim: 0.7754
Epoch 80/500
 - 0s - loss: 1.1306e-05 - mean_squared_error: 2.4263e-10 - keras_psnr: 34.5830 - keras_ssim: 0.7768
Epoch 81/500
 - 0s - loss: 1.1255e-05 - mean_squared_error: 2.4009e-10 - keras_psnr: 34.

Epoch 145/500
 - 0s - loss: 9.0383e-06 - mean_squared_error: 1.5354e-10 - keras_psnr: 36.5704 - keras_ssim: 0.8443
Epoch 146/500
 - 0s - loss: 9.0142e-06 - mean_squared_error: 1.5274e-10 - keras_psnr: 36.5930 - keras_ssim: 0.8450
Epoch 147/500
 - 0s - loss: 8.9903e-06 - mean_squared_error: 1.5194e-10 - keras_psnr: 36.6159 - keras_ssim: 0.8458
Epoch 148/500
 - 0s - loss: 8.9663e-06 - mean_squared_error: 1.5116e-10 - keras_psnr: 36.6382 - keras_ssim: 0.8465
Epoch 149/500
 - 0s - loss: 8.9421e-06 - mean_squared_error: 1.5041e-10 - keras_psnr: 36.6598 - keras_ssim: 0.8472
Epoch 150/500
 - 0s - loss: 8.9184e-06 - mean_squared_error: 1.4968e-10 - keras_psnr: 36.6809 - keras_ssim: 0.8479
Epoch 151/500
 - 0s - loss: 8.8955e-06 - mean_squared_error: 1.4896e-10 - keras_psnr: 36.7018 - keras_ssim: 0.8486
Epoch 152/500
 - 0s - loss: 8.8725e-06 - mean_squared_error: 1.4823e-10 - keras_psnr: 36.7232 - keras_ssim: 0.8493
Epoch 153/500
 - 0s - loss: 8.8499e-06 - mean_squared_error: 1.4750e-10 - keras_

Epoch 217/500
 - 0s - loss: 7.8759e-06 - mean_squared_error: 1.1965e-10 - keras_psnr: 37.6532 - keras_ssim: 0.8784
Epoch 218/500
 - 0s - loss: 7.8663e-06 - mean_squared_error: 1.1936e-10 - keras_psnr: 37.6640 - keras_ssim: 0.8787
Epoch 219/500
 - 0s - loss: 7.8569e-06 - mean_squared_error: 1.1906e-10 - keras_psnr: 37.6750 - keras_ssim: 0.8790
Epoch 220/500
 - 0s - loss: 7.8476e-06 - mean_squared_error: 1.1878e-10 - keras_psnr: 37.6852 - keras_ssim: 0.8792
Epoch 221/500
 - 0s - loss: 7.8385e-06 - mean_squared_error: 1.1850e-10 - keras_psnr: 37.6953 - keras_ssim: 0.8795
Epoch 222/500
 - 0s - loss: 7.8295e-06 - mean_squared_error: 1.1821e-10 - keras_psnr: 37.7062 - keras_ssim: 0.8797
Epoch 223/500
 - 0s - loss: 7.8206e-06 - mean_squared_error: 1.1792e-10 - keras_psnr: 37.7165 - keras_ssim: 0.8800
Epoch 224/500
 - 0s - loss: 7.8119e-06 - mean_squared_error: 1.1768e-10 - keras_psnr: 37.7256 - keras_ssim: 0.8802
Epoch 225/500
 - 0s - loss: 7.8032e-06 - mean_squared_error: 1.1741e-10 - keras_

Epoch 289/500
 - 0s - loss: 7.4018e-06 - mean_squared_error: 1.0492e-10 - keras_psnr: 38.2240 - keras_ssim: 0.8910
Epoch 290/500
 - 0s - loss: 7.3970e-06 - mean_squared_error: 1.0481e-10 - keras_psnr: 38.2287 - keras_ssim: 0.8911
Epoch 291/500
 - 0s - loss: 7.3924e-06 - mean_squared_error: 1.0466e-10 - keras_psnr: 38.2347 - keras_ssim: 0.8912
Epoch 292/500
 - 0s - loss: 7.3879e-06 - mean_squared_error: 1.0449e-10 - keras_psnr: 38.2419 - keras_ssim: 0.8913
Epoch 293/500
 - 0s - loss: 7.3833e-06 - mean_squared_error: 1.0438e-10 - keras_psnr: 38.2464 - keras_ssim: 0.8914
Epoch 294/500
 - 0s - loss: 7.3787e-06 - mean_squared_error: 1.0424e-10 - keras_psnr: 38.2520 - keras_ssim: 0.8915
Epoch 295/500
 - 0s - loss: 7.3743e-06 - mean_squared_error: 1.0410e-10 - keras_psnr: 38.2582 - keras_ssim: 0.8917
Epoch 296/500
 - 0s - loss: 7.3699e-06 - mean_squared_error: 1.0399e-10 - keras_psnr: 38.2627 - keras_ssim: 0.8918
Epoch 297/500
 - 0s - loss: 7.3655e-06 - mean_squared_error: 1.0384e-10 - keras_

Epoch 361/500
 - 0s - loss: 7.1383e-06 - mean_squared_error: 9.6990e-11 - keras_psnr: 38.5653 - keras_ssim: 0.8974
Epoch 362/500
 - 0s - loss: 7.1350e-06 - mean_squared_error: 9.6839e-11 - keras_psnr: 38.5720 - keras_ssim: 0.8975
Epoch 363/500
 - 0s - loss: 7.1319e-06 - mean_squared_error: 9.6762e-11 - keras_psnr: 38.5755 - keras_ssim: 0.8976
Epoch 364/500
 - 0s - loss: 7.1290e-06 - mean_squared_error: 9.6702e-11 - keras_psnr: 38.5782 - keras_ssim: 0.8976
Epoch 365/500
 - 0s - loss: 7.1265e-06 - mean_squared_error: 9.6563e-11 - keras_psnr: 38.5844 - keras_ssim: 0.8977
Epoch 366/500
 - 0s - loss: 7.1247e-06 - mean_squared_error: 9.6615e-11 - keras_psnr: 38.5821 - keras_ssim: 0.8977
Epoch 367/500
 - 0s - loss: 7.1224e-06 - mean_squared_error: 9.6386e-11 - keras_psnr: 38.5924 - keras_ssim: 0.8979
Epoch 368/500
 - 0s - loss: 7.1196e-06 - mean_squared_error: 9.6473e-11 - keras_psnr: 38.5885 - keras_ssim: 0.8979
Epoch 369/500
 - 0s - loss: 7.1156e-06 - mean_squared_error: 9.6199e-11 - keras_

Epoch 433/500
 - 0s - loss: 6.9560e-06 - mean_squared_error: 9.1558e-11 - keras_psnr: 38.8156 - keras_ssim: 0.9017
Epoch 434/500
 - 0s - loss: 6.9549e-06 - mean_squared_error: 9.1614e-11 - keras_psnr: 38.8129 - keras_ssim: 0.9017
Epoch 435/500
 - 0s - loss: 6.9543e-06 - mean_squared_error: 9.1440e-11 - keras_psnr: 38.8212 - keras_ssim: 0.9018
Epoch 436/500
 - 0s - loss: 6.9540e-06 - mean_squared_error: 9.1619e-11 - keras_psnr: 38.8127 - keras_ssim: 0.9018
Epoch 437/500
 - 0s - loss: 6.9531e-06 - mean_squared_error: 9.1356e-11 - keras_psnr: 38.8252 - keras_ssim: 0.9019
Epoch 438/500
 - 0s - loss: 6.9510e-06 - mean_squared_error: 9.1547e-11 - keras_psnr: 38.8161 - keras_ssim: 0.9018
Epoch 439/500
 - 0s - loss: 6.9473e-06 - mean_squared_error: 9.1225e-11 - keras_psnr: 38.8314 - keras_ssim: 0.9020
Epoch 440/500
 - 0s - loss: 6.9435e-06 - mean_squared_error: 9.1318e-11 - keras_psnr: 38.8270 - keras_ssim: 0.9020
Epoch 441/500
 - 0s - loss: 6.9404e-06 - mean_squared_error: 9.1087e-11 - keras_